<a href="https://colab.research.google.com/github/herickcorrea/senna-framework/blob/main/fast_stable_diffusion_AUTOMATIC1111-AJUSTADO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab Pro notebook from https://github.com/TheLastBen/fast-stable-diffusion**

In [20]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth):
    inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(inf)

Shared_Drive = ""  # @param {type:"string"}
# @markdown - Leave empty if you're not using a shared drive

print("🔄 Verificando montagem do Google Drive...")

# Desmontar Google Drive se já estiver montado
if os.path.exists("/content/gdrive"):
    print("⚠️ O Google Drive já está montado. Desmontando...")
    !fusermount -u /content/gdrive
    !rm -rf /content/gdrive

# Montar Google Drive novamente
print("🔗 Conectando ao Google Drive...")
drive.mount('/content/gdrive', force_remount=True)

# Definir caminho principal do Drive
if Shared_Drive != "" and os.path.exists("/content/gdrive/Shareddrives"):
    mainpth = "Shareddrives/" + Shared_Drive
else:
    mainpth = "MyDrive"

clear_output()
inf("✔ Google Drive conectado!", "success", "200px")


Button(button_style='success', description='✔ Google Drive conectado!', disabled=True, layout=Layout(min_width…

In [1]:
#@markdown # Install/Update AUTOMATIC1111 repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time
import base64
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import six


blsaphemy=base64.b64decode(("ZWJ1aQ==").encode('ascii')).decode('ascii')

if not os.path.exists("/content/gdrive"):
  print('[1;31mGdrive not connected, using temporary colab storage ...')
  time.sleep(4)
  mainpth="MyDrive"
  !mkdir -p /content/gdrive/$mainpth
  Shared_Drive=""

if Shared_Drive!="" and not os.path.exists("/content/gdrive/Shareddrives"):
  print('[1;31mShared drive not detected, using default MyDrive')
  mainpth="MyDrive"

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  !git clone -q --depth 1 --branch main https://github.com/TheLastBen/diffusers
  %mkdir -p /content/gdrive/$mainpth/sd
  %cd /content/gdrive/$mainpth/sd
  !git clone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-w$blsaphemy
  !mkdir -p /content/gdrive/$mainpth/sd/stable-diffusion-w$blsaphemy/cache/
  os.environ['TRANSFORMERS_CACHE']=f"/content/gdrive/{mainpth}/sd/stable-diffusion-w"+blsaphemy+"/cache"
  os.environ['TORCH_HOME'] = f"/content/gdrive/{mainpth}/sd/stable-diffusion-w"+blsaphemy+"/cache"
  !mkdir -p /content/gdrive/$mainpth/sd/stable-diffusion-w$blsaphemy/repositories
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-w$blsaphemy-assets /content/gdrive/$mainpth/sd/stable-diffusion-w$blsaphemy/repositories/stable-diffusion-webui-assets

with capture.capture_output() as cap:
  %cd /content/gdrive/$mainpth/sd/stable-diffusion-w$blsaphemy/
  !git reset --hard
  !git checkout master
  time.sleep(1)
  !rm webui.sh
  !git pull
clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [2]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !pip uninstall wandb -y
  if not os.path.exists('/content/gdrive/'+mainpth+'/sd/stablediffusion'):
    !tar -C /content/gdrive/$mainpth --zstd -xf sd_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  if not os.path.exists('gdrive/'+mainpth+'/sd/libtcmalloc/libtcmalloc_minimal.so.4'):
    %env CXXFLAGS=-std=c++14
    !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
    !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch
    %cd /content/gperftools
    !patch -p1 < /content/Patch
    !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
    !mkdir -p /content/gdrive/$mainpth/sd/libtcmalloc && cp .libs/libtcmalloc*.so* /content/gdrive/$mainpth/sd/libtcmalloc
    %env LD_PRELOAD=/content/gdrive/$mainpth/sd/libtcmalloc/libtcmalloc_minimal.so.4
    %cd /content
    !rm *.tar.gz Patch && rm -r /content/gperftools
  else:
    %env LD_PRELOAD=/content/gdrive/$mainpth/sd/libtcmalloc/libtcmalloc_minimal.so.4

  !pip install wandb==0.15.12 pydantic==1.10.2 controlnet_aux -qq
  !pip install diffusers accelerate -U -qq
  !rm -r /usr/local/lib/python3.11/dist-packages/tensorflow*
  os.environ['PYTHONWARNINGS'] = 'ignore'
  !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.11/warnings.py
  !sed -i 's@raise AttributeError(f"module {module!r} has no attribute {name!r}")@@g' /usr/local/lib/python3.11/dist-packages/jax/_src/deprecations.py

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [10]:
#@markdown # Model Download/Load

import requests
from urllib.parse import urlparse, parse_qs
from urllib.request import urlopen, Request
import os
import gdown
import re
from tqdm import tqdm
import six
from urllib.parse import unquote
from IPython.display import clear_output

Use_Temp_Storage = True  # @param {type:"boolean"}
# @markdown - If not, make sure you have enough space on your gdrive

# @markdown ---

Model_Version = "SDXL"  # @param ["SDXL", "1.5", "v1.5 Inpainting", "V2.1-768px"]

# @markdown Or
PATH_to_MODEL = ""  # @param {type:"string"}
# @markdown - Insert the full path of your custom model or to a folder containing multiple models

# @markdown Or
MODEL_LINK = "https://civitai.com/api/download/models/1454015?type=Model&format=SafeTensor&size=pruned&fp=fp16"  # @param {type:"string"}


def getsrc(url):
    parsed_url = urlparse(url)
    if parsed_url.netloc == 'civitai.com':
        return 'civitai'
    elif parsed_url.netloc == 'drive.google.com':
        return 'gdrive'
    elif parsed_url.netloc == 'huggingface.co':
        return 'huggingface'
    return 'others'


src = getsrc(MODEL_LINK)


def get_name(url, gdrive):
    if not gdrive:
        response = requests.get(url, allow_redirects=False)
        if "Location" in response.headers:
            redirected_url = response.headers["Location"]
            quer = parse_qs(urlparse(redirected_url).query)
            if "response-content-disposition" in quer:
                disp_val = quer["response-content-disposition"][0].split(";")
                for vals in disp_val:
                    if vals.strip().startswith("filename="):
                        filenm = unquote(vals.split("=", 1)[1].strip())
                        return filenm.replace("\"", "")
    else:
        headers = {"User-Agent": "Mozilla/5.0"}
        lnk = f"https://drive.google.com/uc?id={url[url.find('/d/')+3:url.find('/view')]}&export=download"
        res = requests.session().get(lnk, headers=headers, stream=True, verify=True)
        res = requests.session().get(get_url_from_gdrive_confirmation(res.text), headers=headers, stream=True, verify=True)
        content_disposition = six.moves.urllib_parse.unquote(res.headers["Content-Disposition"])
        filenm = re.search('attachment; filename="(.*?)"', content_disposition).groups()[0]
        return filenm


def dwn(url, dst, msg):
    file_size = None
    req = Request(url, headers={"User-Agent": "torch.hub"})
    u = urlopen(req)
    meta = u.info()
    content_length = meta.get_all("Content-Length") if hasattr(meta, 'get_all') else meta.getheaders("Content-Length")
    if content_length and len(content_length) > 0:
        file_size = int(content_length[0])

    with tqdm(total=file_size, disable=False, mininterval=0.5, bar_format=msg + ' |{bar:20}| {percentage:3.0f}%') as pbar:
        with open(dst, "wb") as f:
            while True:
                buffer = u.read(8192)
                if len(buffer) == 0:
                    break
                f.write(buffer)
                pbar.update(len(buffer))


if PATH_to_MODEL:
    if os.path.exists(PATH_to_MODEL):
        print("✔ Using the trained model.")
    else:
        while not os.path.exists(PATH_to_MODEL):
            print("✘ Wrong path, use the Colab file explorer to copy the correct path:")
            PATH_to_MODEL = input()
        print("✔ Using the custom model.")

    model = PATH_to_MODEL

elif MODEL_LINK:
    if src == 'civitai':
        modelname = get_name(MODEL_LINK, False)
        model_dir = "/content/temp_models" if Use_Temp_Storage else "/content/gdrive/MyDrive/sd_models"
        os.makedirs(model_dir, exist_ok=True)
        model = os.path.join(model_dir, modelname)

        if not os.path.exists(model):
            dwn(MODEL_LINK, model, "Downloading the custom model")
            clear_output()
            print("✔ Model downloaded successfully.")
        else:
            print("✔ Model already exists.")

    elif src == 'gdrive':
        modelname = get_name(MODEL_LINK, True)
        model_dir = "/content/temp_models" if Use_Temp_Storage else "/content/gdrive/MyDrive/sd_models"
        os.makedirs(model_dir, exist_ok=True)
        model = os.path.join(model_dir, modelname)

        if not os.path.exists(model):
            gdown.download(url=MODEL_LINK, output=model, quiet=False, fuzzy=True)
            clear_output()
            print("✔ Model downloaded successfully.")
        else:
            print("✔ Model already exists.")

    else:
        modelname = os.path.basename(MODEL_LINK)
        model_dir = "/content/temp_models" if Use_Temp_Storage else "/content/gdrive/MyDrive/sd_models"
        os.makedirs(model_dir, exist_ok=True)
        model = os.path.join(model_dir, modelname)

        if not os.path.exists(model):
            gdown.download(url=MODEL_LINK, output=model, quiet=False, fuzzy=True)
            clear_output()
            print("✔ Model downloaded successfully.")
        else:
            print("✔ Model already exists.")

    if os.path.exists(model) and os.path.getsize(model) > 1810671599:
        print("✔ Model downloaded, using the custom model.")
    else:
        os.remove(model)
        print("✘ Wrong link, check that the link is valid.")

# @markdown ---


✔ Model already exists.
✔ Model downloaded, using the custom model.


In [11]:
import os
import requests
from IPython.display import clear_output

# Definir caminho base do Google Drive
mainpth = "MyDrive"  # Nome do diretório no Google Drive
blsaphemy = ""  # Pode ser vazio se não for necessário

# Link do LoRA desejado
LoRA_LINK = "https://civitai.com/api/download/models/97490?type=Model&format=SafeTensor&size=pruned&fp=fp16"

# Criar diretório onde o LoRA será salvo
lora_dir = f"/content/gdrive/{mainpth}/sd/stable-diffusion/models/Lora"
os.makedirs(lora_dir, exist_ok=True)

# Função para baixar o LoRA
def download_lora(url, save_path):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers, stream=True)

    if response.status_code == 200:
        with open(save_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        clear_output()
        print(f"✔ LoRA baixado com sucesso: {save_path}")
    else:
        print("✘ Erro ao baixar o LoRA. Verifique o link.")

# Definir nome do arquivo LoRA e caminho de destino
lora_filename = os.path.basename(LoRA_LINK.split("?")[0])
lora_path = os.path.join(lora_dir, lora_filename)

# Baixar o arquivo se ainda não existir
if not os.path.exists(lora_path):
    download_lora(LoRA_LINK, lora_path)
else:
    print("✔ O LoRA já foi baixado anteriormente.")


✔ LoRA baixado com sucesso: /content/gdrive/MyDrive/sd/stable-diffusion/models/Lora/97490


In [16]:
#@markdown # ControlNet
from torch.hub import download_url_to_file
from urllib.parse import urlparse
import os
import re
from subprocess import run
from IPython.display import clear_output
from IPython.utils import capture

# Definir caminho base do Google Drive
mainpth = "MyDrive"  # Nome do diretório no Google Drive
blsaphemy = ""  # Pode ser vazio se não for necessário

# Modelos disponíveis
XL_Model = "OpenPose"  # @param [ "None", "All", "Canny", "Depth", "Sketch", "OpenPose", "Recolor"]
v1_Model = "Softedge"  # @param [ "None", "All (21GB)", "Canny", "Depth", "Lineart", "MLSD", "Normal", "OpenPose", "Scribble", "Seg", "ip2p", "Shuffle", "Inpaint", "Softedge", "Lineart_Anime", "Tile", "T2iadapter_Models"]
v2_Model = "Depth"  # @param [ "None", "All", "Canny", "Depth", "HED", "OpenPose", "Scribble"]

# Diretório de modelos
mdldir = f"/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/extensions/sd-w{blsaphemy}-controlnet/models"
os.makedirs(mdldir, exist_ok=True)

# Função de download
def download(url, model_dir):
    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print(f'Downloading: {filename}')
        download_url_to_file(url, pth, hash_prefix=None, progress=True)
    else:
        print(f"✔ The model {filename} already exists")

# Links dos modelos ControlNet
model_links = {
    "Canny": 'https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors',
    "Depth": 'https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors',
    "Sketch": 'https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors',
    "OpenPose": 'https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors',
    "Recolor": 'https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_128lora.safetensors',
    "Softedge": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth"
}

# Clonar/Atualizar ControlNet
with capture.capture_output():
    os.chdir(f"/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/extensions")
    if not os.path.exists(f'sd-w{blsaphemy}-controlnet'):
        run(["git", "clone", "https://github.com/Mikubill/sd-w"+blsaphemy+"-controlnet.git"])
    else:
        os.chdir(f'sd-w{blsaphemy}-controlnet')
        run(["git", "reset", "--hard"])
        run(["git", "pull"])

# Ajustar nomes de arquivos nos modelos
for filename in os.listdir(mdldir):
    if "_sd14v1" in filename:
        renamed = re.sub("_sd14v1", "-fp16", filename)
        os.rename(os.path.join(mdldir, filename), os.path.join(mdldir, renamed))

# Baixar listas de modelos disponíveis
os.system("wget -q -O CN_models.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models.txt")
os.system("wget -q -O CN_models_v2.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models_v2.txt")
os.system("wget -q -O CN_models_XL.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models_XL.txt")

with open("CN_models.txt", 'r') as f:
    mdllnk = f.read().splitlines()
with open("CN_models_v2.txt", 'r') as d:
    mdllnk_v2 = d.read().splitlines()
with open("CN_models_XL.txt", 'r') as d:
    mdllnk_XL = d.read().splitlines()

os.remove("CN_models.txt")
os.remove("CN_models_v2.txt")
os.remove("CN_models_XL.txt")

# Download dos modelos conforme a escolha do usuário
if XL_Model != "None":
    if XL_Model == "All":
        for lnk_XL in mdllnk_XL:
            download(lnk_XL, mdldir)
    elif XL_Model in model_links:
        download(model_links[XL_Model], mdldir)
    else:
        print(f"❌ Erro: O modelo '{XL_Model}' não foi encontrado.")
    clear_output()
    print("✔ Done")

if v1_Model != "None":
    if v1_Model == "All (21GB)":
        for lnk in mdllnk:
            download(lnk, mdldir)
    elif v1_Model == "T2iadapter_Models":
        filtered_links = list(filter(lambda x: 't2i' in x, mdllnk))
        for lnk in filtered_links:
            download(lnk, mdldir)
    elif v1_Model in model_links:
        download(model_links[v1_Model], mdldir)
    else:
        print(f"❌ Erro: O modelo '{v1_Model}' não foi encontrado.")
    clear_output()
    print("✔ Done")

if v2_Model != "None":
    if v2_Model == "All":
        for lnk_v2 in mdllnk_v2:
            download(lnk_v2, mdldir)
    elif v2_Model in model_links:
        download(model_links[v2_Model], mdldir)
    else:
        print(f"❌ Erro: O modelo '{v2_Model}' não foi encontrado.")
    clear_output()
    print("✔ Done")


✔ Done


In [ ]:
#@markdown # Start Stable-Diffusion
from IPython.utils import capture
import os
import time
import sys
import fileinput
import re

# Verificar e instalar pyngrok se necessário
try:
    from pyngrok import ngrok, conf
except ModuleNotFoundError:
    print("📥 Instalando pyngrok...")
    !pip install pyngrok
    from pyngrok import ngrok, conf

Ngrok_token = ""  # @param {type:"string"}
# @markdown - Input your ngrok token if you want to use ngrok server

User = ""  # @param {type:"string"}
Password = ""  # @param {type:"string"}
# @markdown - Add credentials to your Gradio interface (optional)

# Configuração de autenticação
auth = f"--gradio-auth {User}:{Password}" if User and Password else ""

# Configurar caminho base do Stable Diffusion no Google Drive
mainpth = "MyDrive"
blsaphemy = ""

# Atualizar arquivos do Stable Diffusion
with capture.capture_output():
    os.chdir(f"/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/modules")
    os.system("wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-w"+blsaphemy+"/master/modules/extras.py")
    os.system("wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-w"+blsaphemy+"/master/modules/sd_models.py")
    os.system("wget -q -O /usr/local/lib/python3.11/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py")
    os.chdir(f"/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/")

    os.system(f"sed -i 's@shared.opts.data[\"sd_model_checkpoint\"] = checkpoint_info.title@shared.opts.data[\"sd_model_checkpoint\"] = checkpoint_info.title;model.half()@' /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/modules/sd_models.py")
    os.system(f"sed -i 's@map_location=\"cpu\"@map_location=\"cuda\"@' /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/modules/extras.py")

    os.system(f"sed -i 's@possible_sd_paths =.*@possible_sd_paths = [\"/content/gdrive/{mainpth}/sd/stablediffusion\"]@' /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/modules/paths.py")
    os.system(f"sed -i 's@../@src/@g' /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/modules/paths.py")
    os.system(f"sed -i 's@src/generative-models@generative-models@g' /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/modules/paths.py")

    os.system(f"sed -i 's@print(\"No module.*@@' /content/gdrive/{mainpth}/sd/stablediffusion/ldm/modules/diffusionmodules/model.py")
    os.system(f"sed -i 's@[\"sd_model_checkpoint\"]@[\"sd_model_checkpoint\", \"sd_vae\", \"CLIP_stop_at_last_layers\", \"inpainting_mask_weight\", \"initial_noise_multiplier\"]@g' /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/modules/shared.py")

# Configuração do ngrok
share = ''
if Ngrok_token:
    try:
        ngrok.kill()
        srv = ngrok.connect(7860, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token), bind_tls=True).public_url

        for line in fileinput.input('/usr/local/lib/python3.11/dist-packages/gradio/blocks.py', inplace=True):
            if line.strip().startswith('self.server_name ='):
                line = f'            self.server_name = "{srv[8:]}"\n'
            if line.strip().startswith('self.protocol = "https"'):
                line = '            self.protocol = "https"\n'
            if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
                line = ''
            if line.strip().startswith('else "http"'):
                line = ''
            sys.stdout.write(line)
    except Exception as e:
        print(f"⚠️ Erro ao iniciar ngrok: {e}")
else:
    share = '--share'

# Configuração de diretório de modelos
ckptdir = ''
if os.path.exists('/content/temp_models'):
    ckptdir = '--ckpt-dir /content/temp_models'

# Verificar se o modelo está carregado corretamente antes de iniciar
try:
    if 'model' in globals() and os.path.exists(model):
        print(f"✔ Iniciando o Stable Diffusion com modelo: {model}")
        os.system(f"python /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/webui.py {share} --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --ckpt \"{model}\" --xformers {auth} --disable-console-progressbars --skip-version-check {ckptdir}")
    else:
        print("⚠️ Modelo não encontrado. Iniciando sem um modelo específico.")
        os.system(f"python /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/webui.py {share} --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --ckpt-dir \"{model}\" --xformers {auth} --disable-console-progressbars --skip-version-check")
except Exception as e:
    print(f"⚠️ Erro ao iniciar o Stable Diffusion: {e}")
    os.system(f"python /content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/webui.py {share} --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers {auth} --disable-console-progressbars --skip-version-check {ckptdir}")
